# Quick tutorial/example, using Env. Data Sci example if possible!

- Okay to incorporate examples found elsewhere (provide link and acknowledgement!)

In [1]:
# import libraries 
import pandas as pd
import numpy as np


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [15]:
# choose data
olympic_athletes = pd.read_csv('../data/Olympic_Athletes/athlete_events.csv')

# make subset of data with only the medaling teams
olympic_subset = olympic_athletes[
    ['Name', 'Team', 'Year', 'Event', 'Medal']
].dropna() # drop rows with na values

# # group the data 
# by_team = olympic_subset.groupby([pd.Grouper(key = 'Year'),
#                                  pd.Grouper(key = 'Medal'),
#                                  pd.Grouper(key = 'Team')]).count()

# by_team.head(10)

# select one country
china_medals = olympic_subset.loc[olympic_subset['Team'] == 'China'].groupby([pd.Grouper(key = 'Year'),
                                                                              pd.Grouper(key = 'Medal')]).count()

china_pivot = china_medals.pivot_table(index = 'Year',
                                       columns = 'Medal',
                                       values = 'Name')

china_medals.head(10)


Name  Team  Event
Year Medal                    
1984 Bronze    37    37     37
     Gold      24    24     24
     Silver    13    13     13
1988 Bronze    30    30     30
     Gold       4     4      4
     Silver    16    16     16
1992 Bronze    15    15     15
     Gold      14    14     14
     Silver    44    44     44
1994 Bronze     2     2      2

In [12]:
# to use the bokeh library start with this line
from bokeh.plotting import figure, show

# create a new plot with a title and axis labels
p = figure(title="Medals Won at the Olympics by China", 
           x_axis_label='Year', 
           y_axis_label='Number of Medals')

# plotting the three lines
p.line(x=china_pivot.index, 
       y=china_pivot.Gold, legend_label=str('Gold'))
p.line(x=china_pivot.index, 
       y=china_pivot.Silver, legend_label=str('Silver'))
p.line(x=china_pivot.index, 
       y=china_pivot.Bronze, legend_label=str('Bronze'))

show(p)


In [20]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import Axis

source = ColumnDataSource(data=dict(year=china_pivot.index, 
                                    gold=china_pivot['Gold'], 
                                    silver=china_pivot['Silver'], 
                                    bronze=china_pivot['Bronze']))

## FH: set tooltips

# figure = china_pivot['Bronze']
# figure.plot(kind='hist', 
#             title='Distribution of Minutes')

# create figure, add tooltips feature
p = figure(title="Medals Won at the Olympics by China", 
           x_axis_label='Year', 
           y_axis_label='Number of Medals',
           x_axis_type = None) # remove defalt x-axis

# add a circle renderer with
# size, color and alpha
p.circle('year', 'gold', 
         size = 10, color = "gold", alpha = 0.5, source=source, legend_label=str('Gold'))
p.circle('year', 'silver', 
         size = 10, color = "silver", alpha = 0.5, source=source, legend_label=str('Silver'))
p.circle('year', 'bronze', 
         size = 10, color = "brown", alpha = 0.5, source=source, legend_label=str('Bronze'))

hover = HoverTool()
hover.tooltips = [
    ("(year,gold)", "(@year, @gold)"),
    ("(year,silver)", "(@year, @silver)"),
    ("(year,bronze)", "(@year, @bronze)"),

]

# Add the hover tool to the plot
p.add_tools(hover)

# change axis ticks
ticker = SingleIntervalTicker(interval = 2, # make year interval on new axis 2
                              num_minor_ticks = 0) # set number of ticks between major ticks
xaxis = LinearAxis(ticker = ticker)
p.add_layout(xaxis, 'below') # identify where new axis should lay
xaxis.formatter.use_scientific = False # turns x axis into non-scientific format


# show the results
show(p) 